In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import to_datetime
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
df = pd.read_csv('/kaggle/input/la-liga-results-19952020/LaLiga_Matches_1995-2021.csv', parse_dates=['Date'], dayfirst=True)

print(df.head())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

    Season       Date    HomeTeam    AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR
0  1995-96 1995-09-02   La Coruna    Valencia     3     0   H   2.0   0.0   H
1  1995-96 1995-09-02    Sp Gijon    Albacete     3     0   H   3.0   0.0   H
2  1995-96 1995-09-03  Ath Bilbao   Santander     4     0   H   2.0   0.0   H
3  1995-96 1995-09-03  Ath Madrid    Sociedad     4     1   H   1.0   1.0   D
4  1995-96 1995-09-03       Celta  Compostela     0     1   A   0.0   0.0   D


In [4]:

def goles_a_favor_ultimos_cinco(equipo, fecha):
    #Comprueba que los últimos 5 partidos jugados por ese equipo pertenezcan a la temporada en cuestión
    #por lo que si le pedimos la predicción un 5 de septiembre solo tendrá en cuenta los partidos jugados hasta el 5 de sept pero a partir del 1 de agosto, 
    #es decir de la temporada nueva que estamos pidiendo
    
    year = int(fecha.strftime("%Y"))
    
    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
            
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01":
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
        
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values('Date', ascending=False)
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    goles = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            goles += partido['FTHG']
        elif partido['AwayTeam'] == equipo:
            goles += partido['FTAG']
    return goles
    
    

def goles_en_contra_ultimos_cinco(equipo, fecha):
    
    year = int(fecha.strftime("%Y")) 
    
    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01":
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values('Date', ascending=False)
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    goles = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            goles += partido['FTAG']
        elif partido['AwayTeam'] == equipo:
            goles += partido['FTHG']
    return goles


def goles_a_favor_descanso_ultimos_cinco(equipo, fecha):
    year = int(fecha.strftime("%Y"))     
    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01": 
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values('Date', ascending=False)
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    goles = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            goles += partido['HTHG']
        elif partido['AwayTeam'] == equipo:
            goles += partido['HTAG']
    return goles


def goles_en_contra_descanso_ultimos_cinco(equipo, fecha):
    year = int(fecha.strftime("%Y"))     
    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01": 
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values('Date', ascending=False)
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    goles = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            goles += partido['HTAG']
        elif partido['AwayTeam'] == equipo:
            goles += partido['HTHG']
    return goles


def puntos_ultimos_cinco(equipo, fecha):
    year = int(fecha.strftime("%Y"))   
    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif fecha.strftime("%Y-%m-%d") >= f"{year}-08-01":
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos_equipo = df[((df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)) & (df['Date'] < fecha) & (df['Season'] == temporada)]
    partidos_ordenados = partidos_equipo.sort_values(['Date'], ascending=[False])
    partidos_ultimos_cinco = partidos_ordenados[(partidos_ordenados['HomeTeam'] == equipo) | (partidos_ordenados['AwayTeam'] == equipo)].head(5)
    puntos = 0
    for i, partido in partidos_ultimos_cinco.iterrows():
        if partido['HomeTeam'] == equipo:
            if(partido['FTR'] == 'H'):
                puntos += 3
            elif(partido['FTR'] == 'D'):
                puntos += 1
            elif(partido['FTR'] == 'A'):
                puntos += 0
        elif partido['AwayTeam'] == equipo:
            
            if(partido['FTR'] == 'A'):
                puntos += 3
            elif(partido['FTR'] == 'D'):
               
                puntos += 1
            elif(partido['FTR'] == 'H'):
               
                puntos += 0
            
    return puntos



In [3]:
def clasificacion_en_una_fecha(fecha):
    year = int(fecha[0:4])
    
    if year == 1999:
        if pd.to_datetime(fecha) >= pd.Timestamp("1999-08-01"):
            temporada = f"{year}-{int(str(year+1)[-4:])}"
        else:
            temporada = f"{year-1}-{int(str(year)[-2:])}"
    elif year == 2000:
        if pd.to_datetime(fecha) <= pd.Timestamp("2000-08-01"):
            temporada = f"{year-1}-{int(str(year)[-4:])}"
        else:
            temporada = f"{year}-{int(str(year+1)[-2:])}"
    elif fecha >= f"{year}-08-01":
        temporada = f"{year}-{int(str(year+1)[-2:])}"
    else:
        temporada = f"{year-1}-{int(str(year)[-2:])}"
    if len(temporada) == 6:
        temporada = f"{temporada[0:4]}-0{temporada[-1]}"
        
    partidos = df[(df['Date'] < fecha) & (df['Season'] == temporada)]
    puntos = {}
    for _, partido in partidos.iterrows():
        if partido['HomeTeam'] not in puntos:
            puntos[partido['HomeTeam']] = 0
        if partido['AwayTeam'] not in puntos:
            puntos[partido['AwayTeam']] = 0
        if partido['FTR'] == 'D':
            puntos[partido['HomeTeam']] += 1
            puntos[partido['AwayTeam']] += 1
        elif partido['FTR'] == 'H':
            puntos[partido['HomeTeam']] += 3
        else:
            puntos[partido['AwayTeam']] += 3
    
    clasificacion = sorted(puntos.items(), key=lambda x: x[1], reverse=True)
    return clasificacion
clasificacion_en_una_fecha('2003-10-03')

[('Valencia', 13),
 ('La Coruna', 12),
 ('Osasuna', 10),
 ('Real Madrid', 10),
 ('Barcelona', 9),
 ('Villarreal', 9),
 ('Sociedad', 8),
 ('Betis', 8),
 ('Sevilla', 8),
 ('Ath Bilbao', 7),
 ('Valladolid', 7),
 ('Celta', 6),
 ('Santander', 5),
 ('Malaga', 4),
 ('Mallorca', 4),
 ('Ath Madrid', 4),
 ('Albacete', 3),
 ('Murcia', 3),
 ('Zaragoza', 3),
 ('Espanol', 2)]

In [4]:
def posicion_equipo_en_una_fecha(equipo, fecha):
    clasificacion = clasificacion_en_una_fecha(fecha)
    posicion = [i+1 for i, (eq, pts) in enumerate(clasificacion) if eq == equipo]
    if posicion:
        return posicion[0]
    else:
        return -1
    

posicion_equipo_en_una_fecha('Betis', '2000-02-18')





12

In [5]:
columnas = ['Temporada', 'Fecha', 'EquipoLocal', 'GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', 'GolesFav1L5', 'GolesCont1L5', 'EquipoVisitante', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante', 'GolesFav1V5', 'GolesCon1V5']
df_nuevo = pd.DataFrame(columns=columnas)
df_nuevo.head()

,Temporada,Fecha,EquipoLocal,GolesFavL5,GolesConL5,PuntosL5,PosicionLocal,GolesFav1L5,GolesCont1L5,EquipoVisitante,GolesFavV5,GolesConV5,PuntosV5,PosicionVisitante,GolesFav1V5,GolesCon1V5


In [6]:

partidos = df

datos_partidos = []
for _, partido in partidos.iterrows():
    temporada = partido['Season']
    fecha = partido['Date']
    ganador = partido['FTR']
    goles_local = partido['FTHG']
    goles_visitante = partido['FTAG']
    goles_local_primera_parte = partido['HTHG']
    goles_visitante_primera_parte = partido['HTAG']
    
    # EQUIPO LOCAL
    equipo_local = partido['HomeTeam']
    goles_a_favor_local_ultimos_5 = goles_a_favor_ultimos_cinco(equipo_local, fecha)
    goles_en_contra_local_ultimos_5 = goles_en_contra_ultimos_cinco(equipo_local, fecha)
    puntos_local_ultimos_5 = puntos_ultimos_cinco( equipo_local, fecha)
    posicion_local = posicion_equipo_en_una_fecha(equipo_local, fecha.strftime("%Y-%m-%d"))
    goles_a_favor_primera_parte_local_ultimos_5 = goles_a_favor_descanso_ultimos_cinco(equipo_local, fecha)
    goles_en_contra_primera_parte_local_ultimos_5 = goles_en_contra_descanso_ultimos_cinco(equipo_local, fecha)
 
    
    # EQUIPO VISITANTE
    equipo_visitante = partido['AwayTeam']
    goles_a_favor_visitante_ultimos_5 = goles_a_favor_ultimos_cinco(equipo_visitante, fecha)
    goles_en_contra_visitante_ultimos_5 = goles_en_contra_ultimos_cinco(equipo_visitante, fecha)
    puntos_visitante_ultimos_5 = puntos_ultimos_cinco( equipo_visitante, fecha)
    posicion_visitante = posicion_equipo_en_una_fecha(equipo_visitante, fecha.strftime("%Y-%m-%d"))
    goles_a_favor_primera_parte_visitante_ultimos_5 = goles_a_favor_descanso_ultimos_cinco(equipo_visitante, fecha)
    goles_en_contra_primera_parte_visitante_ultimos_5 = goles_en_contra_descanso_ultimos_cinco(equipo_visitante, fecha)

    nueva_fila = {'Temporada': temporada, 'Fecha': fecha,'Ganador': ganador, 'Goles Local': goles_local,'Goles Visitante': goles_visitante, 'Goles Local primera parte': goles_local_primera_parte, 'Goles Visitante primera parte': goles_visitante_primera_parte, 'EquipoLocal': equipo_local, 'GolesFavL5': goles_a_favor_local_ultimos_5, 'GolesConL5': goles_en_contra_local_ultimos_5, 'PuntosL5': puntos_local_ultimos_5, 'PosicionLocal': posicion_local, 'GolesFav1L5': goles_a_favor_primera_parte_local_ultimos_5, 'GolesCont1L5': goles_en_contra_primera_parte_local_ultimos_5,
                  'EquipoVisitante': equipo_visitante, ' GolesFavV5': goles_a_favor_visitante_ultimos_5, 'GolesConV5': goles_en_contra_visitante_ultimos_5, 'PuntosV5': puntos_visitante_ultimos_5, 'PosicionVisitante': posicion_visitante, 'GolesFav1V5': goles_a_favor_primera_parte_visitante_ultimos_5, 'GolesCon1V5': goles_en_contra_primera_parte_visitante_ultimos_5}
    
    df_nuevo = df_nuevo.append(nueva_fila, ignore_index=True)
df_nuevo.to_csv('partidos_caracteristicas.csv', index=False)

df_nuevo.head()


KeyboardInterrupt: 

In [5]:
df_nuevo = pd.read_csv('/kaggle/input/datospartidos/partidos_caracteristicas-6.csv', parse_dates=['Fecha'], dayfirst=True)

# Teniendo en cuenta que cada temporada tiene 380 partidos (50 primeras filas de cada temporada)
filas_a_eliminar = []
for i in range(25):
    inicio_temporada = i * 380
    fin_temporada = inicio_temporada + 50
    filas_temporada = list(range(inicio_temporada, fin_temporada))
    filas_a_eliminar.extend(filas_temporada)
    
df_nuevo = df_nuevo.drop(filas_a_eliminar)


In [8]:
df_nuevo.head()
df_nuevo.to_csv('partidos_caracteristicas_LIMPIO.csv', index=False)

In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

# Cargar el archivo CSV original
df = pd.read_csv("/kaggle/input/datases-limpio/partidos_caracteristicas_LIMPIO-2.csv")
df['Ganador'] = df['Ganador'].map({'D': 0, 'H': 1, 'A': 2})  # Convertir las etiquetas de Ganador a tipo float

# Entrenar el modelo con todos los partidos anteriores
X = df[['GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]
y = df['Ganador']

#X = pd.get_dummies(X)  # Convertir las variables categóricas en variables dummy
y = np.array(y)

model = LogisticRegression(solver='liblinear')
model.fit(X, y)

# Filtrar los partidos de la temporada 2020-2021
season_2020_2021 = df[df['Temporada'] == '2020-21']

# Seleccionar las columnas de interés
X_pred = season_2020_2021[['GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]

#X_pred = pd.get_dummies(X_pred)  # Convertir las variables categóricas en variables dummy

# Realizar predicciones para la temporada 2020-2021
pred = model.predict_proba(X_pred)

# Crear un nuevo DataFrame con las columnas deseadas
output_df = pd.DataFrame({
    'Fecha': season_2020_2021['Fecha'],
    'EquipoLocal': season_2020_2021['EquipoLocal'],
    'EquipoVisitante': season_2020_2021['EquipoVisitante'],
    'Prob_Victoria_Local': (pred[:, 1] * 100).round(2).astype(int),  # Probabilidad de victoria del local
    'Prob_Empate': (pred[:, 0] * 100).round(2).astype(int),          # Probabilidad de empate
    'Prob_Victoria_Visitante': (pred[:, 2] * 100).round(2).astype(int)  # Probabilidad de victoria del visitante
})


# Guardar el DataFrame en un nuevo archivo CSV
#output_df.to_csv('predicciones_temporada_2020_2021.csv', index=False)
output_df.head()


,Fecha,EquipoLocal,EquipoVisitante,Prob_Victoria_Local,Prob_Empate,Prob_Victoria_Visitante
8414,2020-09-12,Eibar,Celta,42,25,32
8415,2020-09-12,Granada,Ath Bilbao,42,25,32
8416,2020-09-12,Cadiz,Osasuna,42,25,32
8417,2020-09-13,Alaves,Betis,42,25,32
8418,2020-09-13,Valladolid,Sociedad,42,25,32


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error


df = pd.read_csv("/kaggle/input/datospartidos/partidos_caracteristicas-6.csv")

X = df[['GolesFavL5','GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]
y = df['Goles Local']
y2 = df['Goles Visitante']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("MSE goles local LinearRegression:", mse)




X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, test_size=0.2, random_state=42)

regressor2 = LinearRegression()
regressor2.fit(X_train2, y_train2)
y_pred2 = regressor2.predict(X_test2)

mse = mean_squared_error(y_test2, y_pred2)
print("")
print("MSE goles visitante LinearRegression:", mse)

season_2020_2021 = df[df['Temporada'] == '2020-21']
Z_pred = season_2020_2021[['GolesFavL5', 'GolesConL5', 'PuntosL5', 'PosicionLocal', ' GolesFavV5', 'GolesConV5', 'PuntosV5', 'PosicionVisitante']]
y_pred_local = np.round(regressor.predict(Z_pred)).astype(int)
y_pred_visitante = np.round(regressor2.predict(Z_pred)).astype(int)
output_df['golesLocalFinal'] = y_pred_local
output_df['golesVisitanteFinal'] = y_pred_visitante
output_df.head()

MSE goles local LinearRegression: 1.6140039587423565

MSE goles visitante LinearRegression: 1.1006954277328418


,Fecha,EquipoLocal,EquipoVisitante,Prob_Victoria_Local,Prob_Empate,Prob_Victoria_Visitante,golesLocalFinal,golesVisitanteFinal
8414,2020-09-12,Eibar,Celta,42,25,32,1,1
8415,2020-09-12,Granada,Ath Bilbao,42,25,32,1,1
8416,2020-09-12,Cadiz,Osasuna,42,25,32,1,1
8417,2020-09-13,Alaves,Betis,42,25,32,1,1
8418,2020-09-13,Valladolid,Sociedad,42,25,32,1,1


In [9]:
df = df.dropna() #elimina filas con valores nulos
X = df[['GolesFav1L5','GolesCont1L5', 'PuntosL5', 'PosicionLocal', 'GolesFav1V5', 'GolesCon1V5', 'PuntosV5', 'PosicionVisitante']]
y = df['Goles Local primera parte']
y2 = df['Goles Visitante primera parte']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("MSE goles local 1parte:", mse)

X_train, X_test, y_train2, y_test2 = train_test_split(X, y2, test_size=0.2, random_state=42)

regressor2 = LinearRegression()
regressor2.fit(X_train, y_train2)
y_pred2 = regressor2.predict(X_test)

mse = mean_squared_error(y_test2, y_pred2)
print("MSE goles visitante 1parte:", mse)

season_2020_2021 = df[df['Temporada'] == '2020-21']
Z_pred = season_2020_2021[['GolesFav1L5','GolesCont1L5', 'PuntosL5', 'PosicionLocal', 'GolesFav1V5', 'GolesCon1V5', 'PuntosV5', 'PosicionVisitante']]
y_pred_local = np.round(regressor.predict(Z_pred),).astype(int)
y_pred_visitante = np.round(regressor2.predict(Z_pred)).astype(int)
output_df['golesLocalDescanso'] = y_pred_local
output_df['golesVisitanteDescanso'] = y_pred_visitante
output_df.head()


MSE goles local 1parte: 0.6436545460546695
MSE goles visitante 1parte: 0.4941307451931565


,Fecha,EquipoLocal,EquipoVisitante,Prob_Victoria_Local,Prob_Empate,Prob_Victoria_Visitante,golesLocalFinal,golesVisitanteFinal,golesLocalDescanso,golesVisitanteDescanso
8414,2020-09-12,Eibar,Celta,42,25,32,1,1,1,1
8415,2020-09-12,Granada,Ath Bilbao,42,25,32,1,1,1,1
8416,2020-09-12,Cadiz,Osasuna,42,25,32,1,1,1,1
8417,2020-09-13,Alaves,Betis,42,25,32,1,1,1,1
8418,2020-09-13,Valladolid,Sociedad,42,25,32,1,1,1,1


In [10]:
output_df.to_csv('predicciones_temporada_2020_2021.csv', index=False)
